In [ ]:
import bql
import pandas as pd
import datetime as datetime
from collections import OrderedDict
# Access to dedicated vizualisation libraries 
import bqviz as bqv 
# Access to open source vizualisation libraries 
import plotly.express as px

bq = bql.Service()

In [ ]:
fut_ticker = 'SPX Index'
put_call = 'CALL'
expiry = datetime.datetime(2024,4,5)
rate = 'USDR1T Curncy'
op_ticker = bq.univ.options(fut_ticker)
expiry_str = expiry.strftime("%Y-%m-%d")

price = bq.data.px_last(fill='PREV')
open_interest = bq.data.open_int(fill='PREV')
strike = bq.data.strike_px()
implied_vol = bq.data.IVOL()
moneyness = bq.data.pct_moneyness()
delta = bq.data.DELTA()
vega = bq.data.vega()
gamma = bq.data.GAMMA()
theta = bq.data.THETA()

put_call_cond = bq.data.put_call() == put_call
expiry_cond = bq.data.expire_dt() == expiry_str
conditions = bq.func.and_(put_call_cond, expiry_cond)
op_data_filter = bq.univ.filter(op_ticker, conditions)

items = OrderedDict()
items['Strike'] = strike
items['Moneyness'] = moneyness
items['Opt price'] = price
items['Implied Volatility'] = implied_vol
items['Open Interest'] = open_interest
items['Delta'] = delta
items['Vega'] = vega
items['Gamma'] = gamma
items['Theta'] = theta

req = bql.Request(op_data_filter, items)
res = bq.execute(req)
data = pd.DataFrame({r.name:r.df()[r.name] for r in res})
data.sort_values(by=['Strike'], inplace=True)

In [ ]:
data = data[(data.Moneyness > 98) & (data.Moneyness < 102)]
data

In [ ]:
fig = px.scatter(data, x='Moneyness', y='Implied Volatility', trendline='lowess',
                title= 'IV Vol Surface. Contract: {}, Maturity: {}'.format(fut_ticker, expiry_str),
                template = 'plotly_dark')

fig.show()